# Generate mni subject

1. Imports
2. Download mni template
3. Apply reconall
4. Build surfaces from aseg

## 1. Import libraries

In [4]:
from nipype import config
cfg = dict(logging=dict(workflow_level = 'DEBUG'),
           execution={'stop_on_first_crash': False,
                      'hash_method': 'content'})
config.update_config(cfg)

from nilearn.datasets import load_mni152_template,get_data_dirs, fetch_icbm152_2009,fetch_atlas_talairach
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function
import nipype.interfaces.freesurfer as fs
from nipype.interfaces.io import FreeSurferSource

import nibabel as nib
import numpy as np

import requests
import os

180723-15:21:53,141 duecredit ERROR:
	 Failed to import duecredit due to No module named 'duecredit'


## 2. Download template and parcellations

In [24]:
# Download mni template
data_dir = './mni-template/'
template = fetch_icbm152_2009(data_dir=data_dir)
print(template)

# Download brodmann areas from MRIcron github
url = 'https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/brodmann.nii.lut'
r = requests.get(url)
open(url.split('/')[-1],'w').write(os.path.join(data_dir,url.rsplit('/')[-1]))
url = 'https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/brodmann.nii.gz'
r = requests.get(url)
open(url.split('/')[-1],'w').write(os.path.join(data_dir,url.rsplit('/')[-1]))

# Download aal areas from MRIcron github
url = 'https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/aal.nii.gz'
r = requests.get(url)
open(url.split('/')[-1],'w').write(os.path.join(data_dir,url.rsplit('/')[-1]))
url = 'https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/aal.nii.lut'
r = requests.get(url)
open(url.split('/')[-1],'w').write(os.path.join(data_dir,url.rsplit('/')[-1]))

{'description': b'ICBM 152 Atlas\n\n\nNotes\n-----\nReferences atlases intended for spatial normalization, registration, and\nsegmentation of MRI data. These atlases were generated from large ensembles\nof MRI data in an unbiased fashion. MRI data from templates are derived from\n"a truly normal, well characterized population."\n\n\nContent\n-------\n    :\'csf\': probabilistic map for cerebrospinal fluid\n    :\'gm\': probabilistic map for gray matter\n    :\'wm\': probabilistic map for white matter\n    :\'pd\': proton density-weighted anatomical templates (non-linear average)\n    :\'t1\': T1-weighted anatomical template (non-linear average)\n    :\'t2\': T2-weighted anatomical template (non-linear average)\n    :\'t2_relax\': T2-weighted/relaxed anatomical template\n    :\'eye_mask\': average mask for eyes\n    :\'face_mask\': average mask for face\n    :\'mask\': average mask for brain\n\n\nReferences\n----------\nFor more information see:\nhttp://www.bic.mni.mcgill.ca/ServicesAtl

26

## 3. Apply FreeSurfer's recon-all to template

In [ ]:
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

reconall = ReconAll()
reconall.inputs.subject_id = 'mni'
reconall.inputs.directive = 'all'
reconall.inputs.subjects_dir = data_dir
reconall.inputs.T1_files = template['t1']
reconall.inputs.T2_file = template['t2']
reconall.inputs.use_T2 = True
reconall.inputs.openmp = 2
reconall.inputs.parallel = True
reconall.run()

### Generate surfaces from aseg

In [ ]:
NAMELIST = [
#[1,   'Left-Cerebral-Exterior',                  70,  130, 180, 0],
[2,   'Left-Cerebral-White-Matter',              245, 245, 245, 0],
[3,   'Left-Cerebral-Cortex',                    205, 62,  78,  0],
[4,   'Left-Lateral-Ventricle',                  120, 18,  134, 0],
[5,   'Left-Inf-Lat-Vent',                       196, 58,  250, 0],
#[6,   'Left-Cerebellum-Exterior',                0,   148, 0,   0],
[7,   'Left-Cerebellum-White-Matter',            220, 248, 164, 0],
[8,   'Left-Cerebellum-Cortex',                 230, 148, 34,  0],
[10,  'Left-Thalamus-Proper',                   0,   118, 14,  0],
[11,  'Left-Caudate',                            122, 186, 220, 0],
[12,  'Left-Putamen',                            236, 13,  176, 0],
[13,  'Left-Pallidum',                           12,  48,  255, 0],
[14,  '3rd-Ventricle',                           204, 182, 142, 0],
[15,  '4th-Ventricle',                           42,  204, 164, 0],
[16,  'Brain-Stem',                              119, 159, 176, 0],
[17,  'Left-Hippocampus',                        220, 216, 20,  0],
[18,  'Left-Amygdala',                           103, 255, 255, 0],
#[19,  'Left-Insula',                             80,  196, 98,  0],
[26,  'Left-Accumbens-area',                     255, 165, 0,   0],
[28,  'Left-VentralDC',                          165, 42,  42,  0],
#[40,  'Right-Cerebral-Exterior',                 70,  130, 180, 0],
[41,  'Right-Cerebral-White-Matter',             245, 245, 245, 0],
[42,  'Right-Cerebral-Cortex',                   205, 62,  78, 0],
[43,  'Right-Lateral-Ventricle',                 120, 18,  134, 0],
[44,  'Right-Inf-Lat-Vent',                      196, 58,  250, 0],
#[45,  'Right-Cerebellum-Exterior',               0,   148, 0,   0],
[46,  'Right-Cerebellum-White-Matter',           220, 248, 164, 0],
[47,  'Right-Cerebellum-Cortex',                 230, 148, 34,  0],
[49, 'Right-Thalamus-Proper',                  0,   118, 14,  0],
[50,  'Right-Caudate',                           122, 186, 220, 0],
[51,  'Right-Putamen',                           236, 13,  176, 0],
[52,  'Right-Pallidum',                          13,  48,  255, 0],
[53,  'Right-Hippocampus',                       220, 216, 20,  0],
[54,  'Right-Amygdala',                          103, 255, 255, 0],
#[55,  'Right-Insula',                            80,  196, 98,  0],
[58,  'Right-Accumbens-area',                    255, 165, 0,   0],
[60,  'Right-VentralDC',                         165, 42,  42,  0],
[251, 'CC_Posterior',                            0,   0,   64,  0],
[252, 'CC_Mid_Posterior',                        0,   0,   112, 0],
[253, 'CC_Central',                              0,   0,   160, 0],
[254, 'CC_Mid_Anterior',                         0,   0,   208, 0],
[255, 'CC_Anterior',                             0,   0,   255, 0]]

In [42]:
lut_filepath = 'FreeSurferColorLUT.txt'
lut_array = np.genfromtxt(lut_filepath,
                          dtype=None,
                          usecols=(0, 1, 2, 3, 4, 5),
                          names=['id', 'name', 'R', 'G', 'B', 'A'],
                          encoding='utf-8')
bad_values = [0, 1, 6, 9, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 45, 48, 55, 56, 57, 59, 61, 64, 65, 66, 67, \
              68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 96, 97, 98, 192,  1000]


# print(bad_values)

# Read aseg to extract the labels
fssource2 = FreeSurferSource(subjects_dir=data_dir,subject_id='mni')
res = fssource2.run()
q = nib.load(res.outputs.aparc_aseg[0])
volume_values = np.unique(q.get_data())
# print(volume_values)
# Delete all labels that are not in the volume
all_lut_idx = range(0,len(lut_array))

good_values_idx = []
for idx, elem in enumerate(lut_array):
    if elem[0] not in bad_values and elem[0] in volume_values and elem[0]<1000:
        good_values_idx.append(idx)

bad_values_idx = np.setdiff1d(all_lut_idx, good_values_idx)
lut_array = np.delete(lut_array, bad_values_idx)
print(lut_array)

[(  2, 'Left-Cerebral-White-Matter', 245, 245, 245, 0)
 (  4, 'Left-Lateral-Ventricle', 120,  18, 134, 0)
 (  5, 'Left-Inf-Lat-Vent', 196,  58, 250, 0)
 (  7, 'Left-Cerebellum-White-Matter', 220, 248, 164, 0)
 (  8, 'Left-Cerebellum-Cortex', 230, 148,  34, 0)
 ( 10, 'Left-Thalamus-Proper',   0, 118,  14, 0)
 ( 11, 'Left-Caudate', 122, 186, 220, 0)
 ( 12, 'Left-Putamen', 236,  13, 176, 0)
 ( 13, 'Left-Pallidum',  12,  48, 255, 0)
 ( 14, '3rd-Ventricle', 204, 182, 142, 0)
 ( 15, '4th-Ventricle',  42, 204, 164, 0)
 ( 16, 'Brain-Stem', 119, 159, 176, 0)
 ( 17, 'Left-Hippocampus', 220, 216,  20, 0)
 ( 18, 'Left-Amygdala', 103, 255, 255, 0)
 ( 26, 'Left-Accumbens-area', 255, 165,   0, 0)
 ( 28, 'Left-VentralDC', 165,  42,  42, 0)
 ( 41, 'Right-Cerebral-White-Matter',   0, 225,   0, 0)
 ( 43, 'Right-Lateral-Ventricle', 120,  18, 134, 0)
 ( 44, 'Right-Inf-Lat-Vent', 196,  58, 250, 0)
 ( 46, 'Right-Cerebellum-White-Matter', 220, 248, 164, 0)
 ( 47, 'Right-Cerebellum-Cortex', 230, 148,  34, 0)
 

In [43]:
wf = Workflow(name="generate_surfaces_aseg")

fssource = Node(interface=FreeSurferSource(subjects_dir=os.path.abspath(data_dir),subject_id='mni'), name='fssource')
pretess = Node(interface=fs.MRIPretess(subjects_dir=os.path.abspath(data_dir)), name='pretess')
tess =  Node(interface=fs.MRITessellate(subjects_dir=os.path.abspath(data_dir)), name='tess')
smooth =  Node(interface=fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir)), name='smooth')
convert = Node(interface=fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True), name='convert')

wf.connect(fssource, 'aseg', pretess, 'in_filled')
wf.connect(fssource, 'norm', pretess, 'in_norm')
wf.connect(pretess, 'out_file', tess, 'in_file')
wf.connect(tess, 'surface', smooth, 'in_file')
wf.connect(smooth, 'surface', convert, 'in_file')

In [45]:
output_folder = os.path.join(os.path.abspath(data_dir), 'mni', 'nipype_surfaces')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for l in lut_array:
    if not os.path.exists(os.path.join(output_folder, l[1]+'.surf.gii')):
        # Pre tessellation
        pretess.inputs.label = l[0]
        pretess.inputs.out_file = os.path.join(output_folder, l[1]+'_filled.mgz')
        pretess.outputs
        # Tesselation
        tess.inputs.label_value = l[0]
        tess.inputs.out_file = os.path.join(output_folder,l[1]+'_nonsmooth' )
        # Smoothing
        smooth.inputs.out_file = os.path.join(output_folder,l[1] )
        # Convert
        convert.inputs.out_file = os.path.join(output_folder, l[1]+'.surf.gii')
        # Run pipeline
        wf.run()      

        os.remove(pretess.inputs.out_file)
        os.remove(tess.inputs.out_file)
        os.remove(smooth.inputs.out_file)

In [51]:
fssource = FreeSurferSource(subjects_dir=os.path.abspath(data_dir),subject_id='mni')
res = fssource.run()
print(output_folder)
pials = res.outputs.pial

left_pial_filename = ''
right_pial_filename = ''
for p in pials:
    if 'rh' in p:
        left_pial_filename = p
    elif 'lh' in p:
        right_pial_filename = p

# Convert rh.pial to Right-Cerebral-Cortex.surf.gii
convert = fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True)
convert.inputs.in_file = left_pial_filename
convert.inputs.out_file = os.path.join(output_folder, 'Left-Cerebral-Cortex.surf.gii')
convert.run()    

# Convert rh.pial to Right-Cerebral-Cortex.surf.gii
convert = fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True)
convert.inputs.in_file = right_pial_filename
convert.inputs.out_file = os.path.join(output_folder, 'Right-Cerebral-Cortex.surf.gii')
convert.run()     

/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/nipype_surfaces
180723-16:40:02,75 interface INFO:
	 stdout 2018-07-23T16:40:02.075000:Converting from tkr to scanner coordinates
180723-16:40:02,76 interface INFO:
	 stdout 2018-07-23T16:40:02.075000:Saving /Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/nipype_surfaces/Left-Cerebral-Cortex.surf.gii as a surface
180723-16:40:04,408 interface INFO:
	 stdout 2018-07-23T16:40:04.408569:Converting from tkr to scanner coordinates
180723-16:40:04,410 interface INFO:
	 stdout 2018-07-23T16:40:04.408569:Saving /Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/nipype_surfaces/Right-Cerebral-Cortex.surf.gii as a surface


### Generate surfaces from brodmann areas

In [34]:
q = nib.load(os.path.join(os.path.abspath(data_dir),'brodmann.nii.gz'))
volume_values = np.unique(q.get_data())
print(volume_values)

[ 0  1  2  3  4  5  6  7  8  9 10 11 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]


In [35]:
output_folder = os.path.join(os.path.abspath(data_dir), 'mni', 'brodmann_surfaces')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
        
for l in volume_values:
    label = 'Brodmann_'+str(l)
    if not os.path.exists(os.path.join(output_folder, label+'.vtk')):
        
        # Pre tessellation
        pretess = fs.MRIPretess(subjects_dir=os.path.abspath(data_dir))
        pretess.inputs.label = l
        pretess.inputs.in_filled = os.path.join(data_dir,'brodmann.nii.gz')
        pretess.inputs.in_norm = os.path.join(data_dir,'brodmann.nii.gz')
        pretess.inputs.out_file = os.path.join(output_folder, label+'_filled.mgz')
        pretess.run()
        # Tesselation
        tess =  fs.MRITessellate(subjects_dir=os.path.abspath(data_dir))
        tess.inputs.label_value = l
        tess.inputs.in_file = pretess.inputs.out_file
        tess.inputs.out_file = os.path.join(output_folder,label+'_nonsmooth' )
        tess.run()
        # Smoothing
        smooth =  fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir))
        smooth.inputs.in_file = tess.inputs.out_file
        smooth.inputs.out_file = os.path.join(output_folder,label )
        smooth.run()
        # Convert
        convert = fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True)
        convert.inputs.in_file = smooth.inputs.out_file
        convert.inputs.out_file = os.path.join(output_folder, label+'.vtk')
        convert.run()      

        os.remove(pretess.inputs.out_file)
        os.remove(tess.inputs.out_file)
        os.remove(smooth.inputs.out_file)

180723-16:09:23,770 interface INFO:
	 stderr 2018-07-23T16:09:23.770054:
180723-16:09:23,777 interface INFO:
	 stderr 2018-07-23T16:09:23.777042:Iteration Number : 1
180723-16:09:23,785 interface INFO:
	 stderr 2018-07-23T16:09:23.785470:pass   1 (xy+):  34 found -  34 modified     |    TOTAL:  34
180723-16:09:23,795 interface INFO:
	 stderr 2018-07-23T16:09:23.795248:pass   2 (xy+):   0 found -  34 modified     |    TOTAL:  34
180723-16:09:23,805 interface INFO:
	 stderr 2018-07-23T16:09:23.805796:pass   1 (xy-):  27 found -  27 modified     |    TOTAL:  61
180723-16:09:23,815 interface INFO:
	 stderr 2018-07-23T16:09:23.815908:pass   2 (xy-):   0 found -  27 modified     |    TOTAL:  61
180723-16:09:23,827 interface INFO:
	 stderr 2018-07-23T16:09:23.827277:pass   1 (yz+):  22 found -  22 modified     |    TOTAL:  83
180723-16:09:23,838 interface INFO:
	 stderr 2018-07-23T16:09:23.838130:pass   2 (yz+):   0 found -  22 modified     |    TOTAL:  83
180723-16:09:23,847 interface INFO:


180723-16:09:24,563 interface INFO:
	 stderr 2018-07-23T16:09:24.563580:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:24,565 interface INFO:
	 stderr 2018-07-23T16:09:24.563580:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:24,580 interface INFO:
	 stderr 2018-07-23T16:09:24.580742:Iteration Number : 3
180723-16:09:24,603 interface INFO:
	 stderr 2018-07-23T16:09:24.602984:pass   1 (++):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:24,626 interface INFO:
	 stderr 2018-07-23T16:09:24.626521:pass   1 (+-):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:24,646 interface INFO:
	 stderr 2018-07-23T16:09:24.646535:pass   1 (--):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:24,660 interface INFO:
	 stderr 2018-07-23T16:09:24.660141:pass   1 (-+):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:24,661 interface INFO:
	 stderr 2018-07-23T16:09:24.660141:
180723-16:09:24,664 interface INFO:
	 st

180723-16:09:28,220 interface INFO:
	 stderr 2018-07-23T16:09:28.220480:pass   1 (++):  23 found -  23 modified     |    TOTAL:  23
180723-16:09:28,242 interface INFO:
	 stderr 2018-07-23T16:09:28.242808:pass   2 (++):   0 found -  23 modified     |    TOTAL:  23
180723-16:09:28,259 interface INFO:
	 stderr 2018-07-23T16:09:28.259643:pass   1 (+-):  26 found -  26 modified     |    TOTAL:  49
180723-16:09:28,276 interface INFO:
	 stderr 2018-07-23T16:09:28.276267:pass   2 (+-):   0 found -  26 modified     |    TOTAL:  49
180723-16:09:28,295 interface INFO:
	 stderr 2018-07-23T16:09:28.295462:pass   1 (--):  11 found -  11 modified     |    TOTAL:  60
180723-16:09:28,313 interface INFO:
	 stderr 2018-07-23T16:09:28.313451:pass   2 (--):   0 found -  11 modified     |    TOTAL:  60
180723-16:09:28,330 interface INFO:
	 stderr 2018-07-23T16:09:28.330190:pass   1 (-+):  16 found -  16 modified     |    TOTAL:  76
180723-16:09:28,331 interface INFO:
	 stderr 2018-07-23T16:09:28.330190:pass

180723-16:09:28,952 interface INFO:
	 stderr 2018-07-23T16:09:28.952829:Iteration Number : 4
180723-16:09:28,974 interface INFO:
	 stderr 2018-07-23T16:09:28.974617:pass   1 (++):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:28,995 interface INFO:
	 stderr 2018-07-23T16:09:28.995017:pass   1 (+-):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:29,15 interface INFO:
	 stderr 2018-07-23T16:09:29.015195:pass   1 (--):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:29,24 interface INFO:
	 stderr 2018-07-23T16:09:29.024373:pass   1 (-+):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:29,25 interface INFO:
	 stderr 2018-07-23T16:09:29.024373:
180723-16:09:29,26 interface INFO:
	 stderr 2018-07-23T16:09:29.024373:Total Number of Modified Voxels = 365 (out of 77580: 0.470482)
180723-16:09:29,111 interface INFO:
	 stdout 2018-07-23T16:09:29.111356:niiRead(): NIFTI_UNITS_UNKNOWN, assuming mm
180723-16:09:29,112 interface INFO:
	 stdout 2018-07-23T16:

180723-16:09:33,902 interface INFO:
	 stderr 2018-07-23T16:09:33.902143:pass   1 (--):  14 found -  14 modified     |    TOTAL:  45
180723-16:09:33,917 interface INFO:
	 stderr 2018-07-23T16:09:33.916971:pass   2 (--):   0 found -  14 modified     |    TOTAL:  45
180723-16:09:33,933 interface INFO:
	 stderr 2018-07-23T16:09:33.933625:pass   1 (-+):  16 found -  16 modified     |    TOTAL:  61
180723-16:09:33,934 interface INFO:
	 stderr 2018-07-23T16:09:33.933625:pass   2 (-+):   0 found -  16 modified     |    TOTAL:  61
180723-16:09:33,947 interface INFO:
	 stderr 2018-07-23T16:09:33.947772:Iteration Number : 2
180723-16:09:33,961 interface INFO:
	 stderr 2018-07-23T16:09:33.961116:pass   1 (xy+):   0 found -   0 modified     |    TOTAL:   0
180723-16:09:33,971 interface INFO:
	 stderr 2018-07-23T16:09:33.970970:pass   1 (xy-):   1 found -   1 modified     |    TOTAL:   1
180723-16:09:33,982 interface INFO:
	 stderr 2018-07-23T16:09:33.982463:pass   2 (xy-):   0 found -   1 modified 

...

180723-16:11:14,433 interface INFO:
	 stderr 2018-07-23T16:11:14.433032:pass   2 (yz-):   0 found -   7 modified     |    TOTAL:  41
180723-16:11:14,484 interface INFO:
	 stderr 2018-07-23T16:11:14.483856:pass   1 (xz+):  15 found -  15 modified     |    TOTAL:  56
180723-16:11:14,544 interface INFO:
	 stderr 2018-07-23T16:11:14.544412:pass   2 (xz+):   0 found -  15 modified     |    TOTAL:  56
180723-16:11:14,555 interface INFO:
	 stderr 2018-07-23T16:11:14.555926:pass   1 (xz-):   8 found -   8 modified     |    TOTAL:  64
180723-16:11:14,557 interface INFO:
	 stderr 2018-07-23T16:11:14.555926:pass   2 (xz-):   0 found -   8 modified     |    TOTAL:  64
180723-16:11:14,576 interface INFO:
	 stderr 2018-07-23T16:11:14.576328:Iteration Number : 1
180723-16:11:14,590 interface INFO:
	 stderr 2018-07-23T16:11:14.590663:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:11:14,605 interface INFO:
	 stderr 2018-07-23T16:11:14.605222:pass   1 (+++):   1 found -   1 modif

180723-16:11:16,162 interface INFO:
	 stdout 2018-07-23T16:11:16.158468:  $Id: mrisurf.c,v 1.781.2.6 2016/12/27 16:47:14 zkaufman Exp $
180723-16:11:16,720 interface INFO:
	 stdout 2018-07-23T16:11:16.720287:slice 70: 1875 vertices, 1946 faces
180723-16:11:16,721 interface INFO:
	 stdout 2018-07-23T16:11:16.720287:slice 80: 3711 vertices, 3781 faces
180723-16:11:16,722 interface INFO:
	 stdout 2018-07-23T16:11:16.720287:slice 90: 6706 vertices, 6825 faces
180723-16:11:16,723 interface INFO:
	 stdout 2018-07-23T16:11:16.720287:slice 100: 10312 vertices, 10423 faces
180723-16:11:16,723 interface INFO:
	 stdout 2018-07-23T16:11:16.720287:slice 110: 14243 vertices, 14330 faces
180723-16:11:16,727 interface INFO:
	 stdout 2018-07-23T16:11:16.720287:slice 120: 16082 vertices, 16076 faces
180723-16:11:16,730 interface INFO:
	 stdout 2018-07-23T16:11:16.720287:slice 130: 16082 vertices, 16076 faces
180723-16:11:16,731 interface INFO:
	 stdout 2018-07-23T16:11:16.720287:slice 140: 16082 vertice

180723-16:11:20,616 interface INFO:
	 stderr 2018-07-23T16:11:20.616912:pass   2 (xz+):   0 found -   2 modified     |    TOTAL:   5
180723-16:11:20,629 interface INFO:
	 stderr 2018-07-23T16:11:20.629624:pass   1 (xz-):   1 found -   1 modified     |    TOTAL:   6
180723-16:11:20,630 interface INFO:
	 stderr 2018-07-23T16:11:20.629624:pass   2 (xz-):   0 found -   1 modified     |    TOTAL:   6
180723-16:11:20,645 interface INFO:
	 stderr 2018-07-23T16:11:20.644952:Iteration Number : 2
180723-16:11:20,656 interface INFO:
	 stderr 2018-07-23T16:11:20.656793:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:11:20,666 interface INFO:
	 stderr 2018-07-23T16:11:20.666055:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:11:20,681 interface INFO:
	 stderr 2018-07-23T16:11:20.681555:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:11:20,682 interface INFO:
	 stderr 2018-07-23T16:11:20.681555:pass   1 (+++):   0 found -   0 modif

180723-16:11:22,681 interface INFO:
	 stdout 2018-07-23T16:11:22.646652:slice 130: 14130 vertices, 14092 faces
180723-16:11:22,682 interface INFO:
	 stdout 2018-07-23T16:11:22.646652:slice 140: 14130 vertices, 14092 faces
180723-16:11:22,683 interface INFO:
	 stdout 2018-07-23T16:11:22.646652:slice 150: 14130 vertices, 14092 faces
180723-16:11:22,685 interface INFO:
	 stdout 2018-07-23T16:11:22.646652:slice 160: 14130 vertices, 14092 faces
180723-16:11:22,687 interface INFO:
	 stdout 2018-07-23T16:11:22.646652:slice 170: 14130 vertices, 14092 faces
180723-16:11:22,697 interface INFO:
	 stdout 2018-07-23T16:11:22.646652:slice 180: 14130 vertices, 14092 faces
180723-16:11:22,698 interface INFO:
	 stdout 2018-07-23T16:11:22.646652:using the conformed surface RAS to save vertex points...
180723-16:11:22,701 interface INFO:
	 stdout 2018-07-23T16:11:22.646652:writing /Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/brodmann_surfaces/Brodmann_47_nonsmooth
180723-16:11:22,70

180723-16:11:26,361 interface INFO:
	 stderr 2018-07-23T16:11:26.361707:Iteration Number : 2
180723-16:11:26,377 interface INFO:
	 stderr 2018-07-23T16:11:26.376985:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:11:26,395 interface INFO:
	 stderr 2018-07-23T16:11:26.395098:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:11:26,407 interface INFO:
	 stderr 2018-07-23T16:11:26.407858:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:11:26,408 interface INFO:
	 stderr 2018-07-23T16:11:26.407858:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-16:11:26,430 interface INFO:
	 stderr 2018-07-23T16:11:26.430815:Iteration Number : 2
180723-16:11:26,450 interface INFO:
	 stderr 2018-07-23T16:11:26.450365:pass   1 (++):   1 found -   1 modified     |    TOTAL:   1
180723-16:11:26,470 interface INFO:
	 stderr 2018-07-23T16:11:26.470394:pass   2 (++):   0 found -   1 modified     |    TOTAL:   1
180723-16:11:26,49

### Generte surfaces from brodmann areas

In [32]:
q = nib.load(os.path.join(os.path.abspath(data_dir),'aal.nii.gz'))
volume_values = np.unique(q.get_data())
print(volume_values)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116]


In [33]:
output_folder = os.path.join(os.path.abspath(data_dir), 'mni', 'aal_surfaces')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
        
for l in volume_values:
    label = 'AAL_'+str(l)
    if not os.path.exists(os.path.join(output_folder, label+'.vtk')):
        
        # Pre tessellation
        pretess = fs.MRIPretess(subjects_dir=os.path.abspath(data_dir))
        pretess.inputs.label = l
        pretess.inputs.in_filled = os.path.join(data_dir,'aal.nii.gz')
        pretess.inputs.in_norm = os.path.join(data_dir,'aal.nii.gz')
        pretess.inputs.out_file = os.path.join(output_folder, label+'_filled.mgz')
        pretess.run()
        # Tesselation
        tess =  fs.MRITessellate(subjects_dir=os.path.abspath(data_dir))
        tess.inputs.label_value = l
        tess.inputs.in_file = pretess.inputs.out_file
        tess.inputs.out_file = os.path.join(output_folder,label+'_nonsmooth' )
        tess.run()
        # Smoothing
        smooth =  fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir))
        smooth.inputs.in_file = tess.inputs.out_file
        smooth.inputs.out_file = os.path.join(output_folder,label )
        smooth.run()
        # Convert
        convert = fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),to_scanner=True)
        convert.inputs.in_file = smooth.inputs.out_file
        convert.inputs.out_file = os.path.join(output_folder, label+'.vtk')
        convert.run()      

        os.remove(pretess.inputs.out_file)
        os.remove(tess.inputs.out_file)
        os.remove(smooth.inputs.out_file)

180723-15:51:20,572 interface INFO:
	 stderr 2018-07-23T15:51:20.572764:
180723-15:51:20,587 interface INFO:
	 stderr 2018-07-23T15:51:20.587218:Iteration Number : 1
180723-15:51:20,604 interface INFO:
	 stderr 2018-07-23T15:51:20.604876:pass   1 (xy+):  78 found -  78 modified     |    TOTAL:  78
180723-15:51:20,621 interface INFO:
	 stderr 2018-07-23T15:51:20.621403:pass   2 (xy+):   0 found -  78 modified     |    TOTAL:  78
180723-15:51:20,640 interface INFO:
	 stderr 2018-07-23T15:51:20.640513:pass   1 (xy-):  19 found -  19 modified     |    TOTAL:  97
180723-15:51:20,659 interface INFO:
	 stderr 2018-07-23T15:51:20.659857:pass   2 (xy-):   0 found -  19 modified     |    TOTAL:  97
180723-15:51:20,679 interface INFO:
	 stderr 2018-07-23T15:51:20.679252:pass   1 (yz+):  48 found -  48 modified     |    TOTAL: 145
180723-15:51:20,696 interface INFO:
	 stderr 2018-07-23T15:51:20.696855:pass   2 (yz+):   0 found -  48 modified     |    TOTAL: 145
180723-15:51:20,716 interface INFO:


180723-15:51:21,580 interface INFO:
	 stderr 2018-07-23T15:51:21.579420:
180723-15:51:21,581 interface INFO:
	 stderr 2018-07-23T15:51:21.579420:Total Number of Modified Voxels = 401 (out of 5629569: 0.007123)
180723-15:51:21,661 interface INFO:
	 stdout 2018-07-23T15:51:21.661401:niiRead(): NIFTI_UNITS_UNKNOWN, assuming mm
180723-15:51:21,662 interface INFO:
	 stdout 2018-07-23T15:51:21.661401:niiRead(): NIFTI_UNITS_UNKNOWN, assuming mm
180723-15:51:21,663 interface INFO:
	 stdout 2018-07-23T15:51:21.661401:Ambiguous edge configurations... 
180723-15:51:21,664 interface INFO:
	 stdout 2018-07-23T15:51:21.661401:
180723-15:51:21,664 interface INFO:
	 stdout 2018-07-23T15:51:21.661401:mri_pretess done
180723-15:51:21,667 interface INFO:
	 stdout 2018-07-23T15:51:21.661401:
180723-15:51:21,944 interface INFO:
	 stdout 2018-07-23T15:51:21.944639:$Id: mri_tessellate.c,v 1.38.2.1 2016/07/26 18:46:38 zkaufman Exp $
180723-15:51:21,946 interface INFO:
	 stdout 2018-07-23T15:51:21.944639:  $Id

180723-15:51:39,194 interface INFO:
	 stderr 2018-07-23T15:51:39.194733:pass   1 (yz-):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:39,205 interface INFO:
	 stderr 2018-07-23T15:51:39.204979:pass   1 (xz+):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:39,206 interface INFO:
	 stderr 2018-07-23T15:51:39.204979:pass   1 (xz-):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:39,217 interface INFO:
	 stderr 2018-07-23T15:51:39.217317:Iteration Number : 2
180723-15:51:39,227 interface INFO:
	 stderr 2018-07-23T15:51:39.227549:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:39,236 interface INFO:
	 stderr 2018-07-23T15:51:39.236889:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:39,245 interface INFO:
	 stderr 2018-07-23T15:51:39.245092:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:39,247 interface INFO:
	 stderr 2018-07-23T15:51:39.245092:pass   1 (+++):   0 found -   0 modif

180723-15:51:42,80 interface INFO:
	 stderr 2018-07-23T15:51:42.080624:pass   1 (--):   0 found -   0 modified     |    TOTAL:   1
180723-15:51:42,94 interface INFO:
	 stderr 2018-07-23T15:51:42.094828:pass   1 (-+):   3 found -   3 modified     |    TOTAL:   4
180723-15:51:42,96 interface INFO:
	 stderr 2018-07-23T15:51:42.094828:pass   2 (-+):   0 found -   3 modified     |    TOTAL:   4
180723-15:51:42,104 interface INFO:
	 stderr 2018-07-23T15:51:42.104587:Iteration Number : 2
180723-15:51:42,116 interface INFO:
	 stderr 2018-07-23T15:51:42.115729:pass   1 (xy+):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:42,126 interface INFO:
	 stderr 2018-07-23T15:51:42.126480:pass   1 (xy-):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:42,135 interface INFO:
	 stderr 2018-07-23T15:51:42.135490:pass   1 (yz+):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:42,144 interface INFO:
	 stderr 2018-07-23T15:51:42.144337:pass   1 (yz-):   0 found -   0 modified   

180723-15:51:44,905 interface INFO:
	 stderr 2018-07-23T15:51:44.904960:Iteration Number : 1
180723-15:51:44,914 interface INFO:
	 stderr 2018-07-23T15:51:44.914491:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:44,925 interface INFO:
	 stderr 2018-07-23T15:51:44.925302:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:44,933 interface INFO:
	 stderr 2018-07-23T15:51:44.933064:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:44,934 interface INFO:
	 stderr 2018-07-23T15:51:44.933064:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:51:44,949 interface INFO:
	 stderr 2018-07-23T15:51:44.949045:Iteration Number : 1
180723-15:51:44,964 interface INFO:
	 stderr 2018-07-23T15:51:44.964713:pass   1 (++):   1 found -   1 modified     |    TOTAL:   1
180723-15:51:44,980 interface INFO:
	 stderr 2018-07-23T15:51:44.980337:pass   2 (++):   0 found -   1 modified     |    TOTAL:   1
180723-15:51:44,99

...

180723-15:56:17,345 interface INFO:
	 stderr 2018-07-23T15:56:17.345132:pass   1 (yz+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:17,353 interface INFO:
	 stderr 2018-07-23T15:56:17.353661:pass   1 (yz-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:17,360 interface INFO:
	 stderr 2018-07-23T15:56:17.360741:pass   1 (xz+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:17,363 interface INFO:
	 stderr 2018-07-23T15:56:17.360741:pass   1 (xz-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:17,372 interface INFO:
	 stderr 2018-07-23T15:56:17.372222:Iteration Number : 2
180723-15:56:17,383 interface INFO:
	 stderr 2018-07-23T15:56:17.383497:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:17,391 interface INFO:
	 stderr 2018-07-23T15:56:17.390999:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:17,400 interface INFO:
	 stderr 2018-07-23T15:56:17.400406:pass   1 (+++):   0 found -   0 modif

180723-15:56:19,491 interface INFO:
	 stderr 2018-07-23T15:56:19.491362:pass   1 (+-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:19,506 interface INFO:
	 stderr 2018-07-23T15:56:19.506098:pass   1 (--):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:19,506 interface INFO:
	 stderr 2018-07-23T15:56:19.506098:pass   1 (-+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:19,514 interface INFO:
	 stderr 2018-07-23T15:56:19.514830:Iteration Number : 2
180723-15:56:19,526 interface INFO:
	 stderr 2018-07-23T15:56:19.526313:pass   1 (xy+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:19,534 interface INFO:
	 stderr 2018-07-23T15:56:19.534282:pass   1 (xy-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:19,542 interface INFO:
	 stderr 2018-07-23T15:56:19.542653:pass   1 (yz+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:19,550 interface INFO:
	 stderr 2018-07-23T15:56:19.550724:pass   1 (yz-):   0 found -   0 modified

180723-15:56:21,598 interface INFO:
	 stderr 2018-07-23T15:56:21.598027:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:21,608 interface INFO:
	 stderr 2018-07-23T15:56:21.608362:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:21,609 interface INFO:
	 stderr 2018-07-23T15:56:21.608362:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:21,626 interface INFO:
	 stderr 2018-07-23T15:56:21.625976:Iteration Number : 1
180723-15:56:21,640 interface INFO:
	 stderr 2018-07-23T15:56:21.639943:pass   1 (++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:21,654 interface INFO:
	 stderr 2018-07-23T15:56:21.654416:pass   1 (+-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:21,668 interface INFO:
	 stderr 2018-07-23T15:56:21.668056:pass   1 (--):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:21,676 interface INFO:
	 stderr 2018-07-23T15:56:21.676580:pass   1 (-+):   0 found -   0 modified 

180723-15:56:23,916 interface INFO:
	 stderr 2018-07-23T15:56:23.916090:Iteration Number : 2
180723-15:56:23,928 interface INFO:
	 stderr 2018-07-23T15:56:23.928668:pass   1 (xy+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:23,936 interface INFO:
	 stderr 2018-07-23T15:56:23.936351:pass   1 (xy-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:23,945 interface INFO:
	 stderr 2018-07-23T15:56:23.945894:pass   1 (yz+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:23,954 interface INFO:
	 stderr 2018-07-23T15:56:23.954158:pass   1 (yz-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:23,964 interface INFO:
	 stderr 2018-07-23T15:56:23.964209:pass   1 (xz+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:23,965 interface INFO:
	 stderr 2018-07-23T15:56:23.964209:pass   1 (xz-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:23,974 interface INFO:
	 stderr 2018-07-23T15:56:23.973857:Iteration Number : 2
180723-15:56:23,

180723-15:56:26,67 interface INFO:
	 stderr 2018-07-23T15:56:26.067395:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:26,68 interface INFO:
	 stderr 2018-07-23T15:56:26.067395:pass   1 (+++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:26,83 interface INFO:
	 stderr 2018-07-23T15:56:26.083444:Iteration Number : 1
180723-15:56:26,98 interface INFO:
	 stderr 2018-07-23T15:56:26.097944:pass   1 (++):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:26,112 interface INFO:
	 stderr 2018-07-23T15:56:26.112546:pass   1 (+-):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:26,126 interface INFO:
	 stderr 2018-07-23T15:56:26.126191:pass   1 (--):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:26,128 interface INFO:
	 stderr 2018-07-23T15:56:26.126191:pass   1 (-+):   0 found -   0 modified     |    TOTAL:   0
180723-15:56:26,135 interface INFO:
	 stderr 2018-07-23T15:56:26.134945:Iteration Number : 2
180723-15:56:26,144 inte